# Generate grounded answers

> use LangChain and Google Cloud + LangChain integrations to orchestrate RAG pipeline

In [2]:
# ! cd .. && tree

## imports

In [3]:
import os, shutil, json

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from IPython.display import Markdown

from google.cloud import aiplatform
import vertexai.generative_models
import vertexai.language_models
import google.cloud.discoveryengine_v1 as discoveryengine

print(f"vertexai SDK version = {vertexai.__version__}")
print(f"AI Platform SDK version = {aiplatform.__version__}")
print(f"Discovery Engine API version = {discoveryengine.__version__}")

vertexai SDK version = 1.71.1
AI Platform SDK version = 1.71.1
Discovery Engine API version = 0.12.2


## config

In [4]:
PREFIX="mortgage-ball"

In [5]:
PROJECT_ID = "hybrid-vertex"
REGION = "us-central1"
LOCATION = REGION.split('-')[0]

EXPERIMENT_1 = "mlb"
EXPERIMENT_2 = "lending"

SERIES = "ranking-api"

# Cloud storage buckets
GCS_BUCKET_URI = f"gs://{PREFIX}-central-bucket"
GCS_BUCKET = GCS_BUCKET_URI.replace("gs://", "")

In [6]:
# Vertex AI
vertexai.init(project = PROJECT_ID, location = REGION)

# Vertex AI Agent Builder APIs
check_grounding_client = discoveryengine.GroundedGenerationServiceClient()

## Get chunked documents

In [7]:
local_dir = "files/embeddings-api"

with open(local_dir+'/chunk-embeddings.jsonl', 'r') as f:
    chunks = [json.loads(line) for line in f]

### Review a chunk

In [8]:
chunks[0].keys()

dict_keys(['instance', 'predictions', 'status'])

In [9]:
chunks[0]['instance']['chunk_id']

'c2'

In [10]:
print(chunks[0]['instance']['content'])

# OFFICIAL BASEBALL RULES

## Official Baseball Rules 2023 Edition

### JOINT COMPETITION COMMITTEE

|-|-|-|
| Bill DeWitt | Whit Merrifield | Austin Slater |
| Jack Flaherty | Bill Miller | John Stanton, Chair |
| Tyler Glasnow | Dick Monfort | Tom Werner |
| Greg Johnson | Mark Shapiro |  |

Committee Secretary Paul V. Mifsud, Jr. Copyright © 2023 by the Office of the Commissioner of Baseball


In [11]:
chunks[0]['predictions'][0]['embeddings']['values'][0:10]

[0.008681542240083218,
 0.06999468058347702,
 0.003673204220831394,
 0.019888797774910927,
 0.016285404562950134,
 0.035664502531290054,
 0.06200747936964035,
 0.05597030743956566,
 0.0034793149679899216,
 -0.024485772475600243]

### prep content chunks for simple RAG

In [12]:
content_chunks = {}
for chunk in chunks:
    content_chunks[chunk['instance']['chunk_id']] = chunk['instance']['content']

In [13]:
content_chunks['c2']

'# OFFICIAL BASEBALL RULES\n\n## Official Baseball Rules 2023 Edition\n\n### JOINT COMPETITION COMMITTEE\n\n|-|-|-|\n| Bill DeWitt | Whit Merrifield | Austin Slater |\n| Jack Flaherty | Bill Miller | John Stanton, Chair |\n| Tyler Glasnow | Dick Monfort | Tom Werner |\n| Greg Johnson | Mark Shapiro |  |\n\nCommittee Secretary Paul V. Mifsud, Jr. Copyright © 2023 by the Office of the Commissioner of Baseball'

## Vector DB With Numpy

> retreiving from local numpy DB for simplicity

In [14]:
vector_db = [
    [
        chunk['instance']['chunk_id'],
        chunk['predictions'][0]['embeddings']['values'],
    ]
    for chunk in chunks
]
vector_index = np.array([row[1] for row in vector_db])

## Models: Embeddings, Generation

In [43]:
embedder = vertexai.language_models.TextEmbeddingModel.from_pretrained('text-embedding-004')
llm = vertexai.generative_models.GenerativeModel("gemini-1.5-flash-001")

In [44]:
question = "What are the dimensions of a base?"

Markdown(llm.generate_content(question).text)

The term "base" is quite ambiguous and can refer to different things depending on the context. To answer your question, I need more information. Please tell me:

**What kind of base are you asking about?**

For example, are you asking about:

* **A base in geometry?** (like the base of a triangle or rectangle)
* **A base in chemistry?** (like a base in an acid-base reaction)
* **A base in a military context?** (like a military base)
* **The base of a number system?** (like base 10, base 2, etc.) 

Once you provide more context, I can give you the specific dimensions you need. 


### get embedding for question

In [17]:
question_embedding = embedder.get_embeddings([question])[0].values
question_embedding[0:10]

[-0.026682045310735703,
 0.011593513190746307,
 0.028523651883006096,
 -0.0017065361607819796,
 0.01946176588535309,
 0.0031198114156723022,
 0.07915323227643967,
 -0.005078596994280815,
 -0.006295712199062109,
 0.04943541809916496]

### Retrieval: Matching With Numpy

In [18]:
similarity = np.dot(question_embedding, vector_index.T)
#matches = np.argsort(similarity)[::-1][:5].tolist()
matches = np.argsort(similarity)[-5:].tolist()
matches = [(match, similarity[match]) for match in matches]
matches.reverse()
matches

[(38, 0.5843799337008113),
 (34, 0.5724333016720691),
 (835, 0.5244194362041271),
 (40, 0.5126844935129918),
 (26, 0.5033481946111171)]

In [19]:
for m, match in enumerate(matches):
    print(f"Match {m+1} ({match[1]:.2f}) is chunk {vector_db[match[0]][0]}:\n{content_chunks[vector_db[match[0]][0]]}\n###################################################")

Match 1 (0.58) is chunk c38:
# 2.00-THE PLAYING FIELD

## 2.02 Home Base

Home base shall be marked by a five-sided slab of whitened rubber. It shall be a 17-inch square with two of the corners removed so that one edge is 17 inches long, two adjacent sides are 8\frac{1}{2} inches and the remaining two sides are 12 inches and set at an angle to make a point.
###################################################
Match 2 (0.57) is chunk c39:
# 2.00-THE PLAYING FIELD

## 2.02 Home Base

It shall be set in the ground with the point at the intersection of the lines extending from home base to first base and to third base; with the 17-inch edge facing the pitcher's plate, and the two 12-inch edges coinciding with the first and third base lines. The top edges of home base shall be beveled and the base shall be fixed in the ground level with the ground surface. (See drawing D in Appendix 2.) 3
###################################################
Match 3 (0.52) is chunk c837:
# APPENDICES

## Appen

# Generation: Q&A With Gemini Grounded With RAG

> Provide the matched chunks of text along with the question as a prompt to a generative model for a grounded answe

### Display Helper Function

In [ ]:
## TODO - abstract to file

from IPython.display import Markdown, display
from vertexai.generative_models import (
    GenerationResponse,
    GenerativeModel,
    Tool,
    grounding,
)

def print_grounding_response(response: GenerationResponse):
    """Prints Gemini response with grounding citations."""
    grounding_metadata = response.candidates[0].grounding_metadata

    # Citation indices are in byte units
    ENCODING = "utf-8"
    text_bytes = response.text.encode(ENCODING)

    prev_index = 0
    markdown_text = ""

    for grounding_support in grounding_metadata.grounding_supports:
        text_segment = text_bytes[
            prev_index : grounding_support.segment.end_index
        ].decode(ENCODING)

        footnotes_text = ""
        for grounding_chunk_index in grounding_support.grounding_chunk_indices:
            footnotes_text += f"[{grounding_chunk_index + 1}]"

        markdown_text += f"{text_segment} {footnotes_text}\n"
        prev_index = grounding_support.segment.end_index

    if prev_index < len(text_bytes):
        markdown_text += str(text_bytes[prev_index:], encoding=ENCODING)

    markdown_text += "\n----\n## Grounding Sources\n"

    if grounding_metadata.web_search_queries:
        markdown_text += (
            f"\n**Web Search Queries:** {grounding_metadata.web_search_queries}\n"
        )
        if grounding_metadata.search_entry_point:
            markdown_text += f"\n**Search Entry Point:**\n {grounding_metadata.search_entry_point.rendered_content}\n"
    elif grounding_metadata.retrieval_queries:
        markdown_text += (
            f"\n**Retrieval Queries:** {grounding_metadata.retrieval_queries}\n"
        )

    markdown_text += "### Grounding Chunks\n"

    for index, grounding_chunk in enumerate(
        grounding_metadata.grounding_chunks, start=1
    ):
        context = grounding_chunk.web or grounding_chunk.retrieved_context
        if not context:
            print(f"Skipping Grounding Chunk {grounding_chunk}")
            continue

        markdown_text += f"{index}. [{context.title}]({context.uri})\n"

    display(Markdown(markdown_text))

### Prompt Building Function

> Use the matching chunks as context for the prompt:

In [20]:
def get_prompt(question, top_n = 5):
    # get embedding for question
    question_embedding = embedder.get_embeddings([question])[0].values
    
    # get top_n matches:
    similarity = np.dot(question_embedding, vector_index.T)
    matches = np.argsort(similarity)[-top_n:].tolist()
    matches.reverse()
    matches = [[match, similarity[match]] for match in matches]
    
    # construct prompt:
    prompt = ''
    for m, match in enumerate(matches):
        prompt += f"Context {m+1}:\n{content_chunks[vector_db[match[0]][0]]}\n\n"
    prompt += f'Answer the following question using the provided contexts:\n{question}'
    
    return matches, prompt

In [21]:
matches, prompt = get_prompt(question) 
print(prompt)

Context 1:
# 2.00-THE PLAYING FIELD

## 2.02 Home Base

Home base shall be marked by a five-sided slab of whitened rubber. It shall be a 17-inch square with two of the corners removed so that one edge is 17 inches long, two adjacent sides are 8\frac{1}{2} inches and the remaining two sides are 12 inches and set at an angle to make a point.

Context 2:
# 2.00-THE PLAYING FIELD

## 2.02 Home Base

It shall be set in the ground with the point at the intersection of the lines extending from home base to first base and to third base; with the 17-inch edge facing the pitcher's plate, and the two 12-inch edges coinciding with the first and third base lines. The top edges of home base shall be beveled and the base shall be fixed in the ground level with the ground surface. (See drawing D in Appendix 2.) 3

Context 3:
# APPENDICES

## Appendix 2

Diagram No. 2 Layout at Home Plate, 1st, 2nd, and 3rd Bases 18" A 18" 90° LAYOUT AT SECOND BASE FOR LAYOUT AT PITCHER'S PLATE SEE DIAGRAM NO. 3 90° 6"

In [46]:
# Markdown(prompt)

## Grounded Generation

In [22]:
answer = llm.generate_content(prompt).text
Markdown(answer)

The dimensions of a base depend on whether it's home base or the other bases. 

* **Home Base:**  It is a five-sided slab with:
    * One edge of 17 inches
    * Two adjacent sides of 8 1/2 inches
    * Two remaining sides of 12 inches.

* **First, Second, and Third Base:** They are white canvas or rubber-covered bags that are:
    * 18 inches square
    * 3 to 5 inches thick. 


## Check Grounding API

With this API you pass it the answer from an LLM along with chunks of context which all called facts in the API

The response include an overall score, support_score, and a phrase by phrase breakdown of which chunk/fact is the citation for the phrase - if any.

see [Discoveryengine Python Grounded Generation Service Client](https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1.services.grounded_generation_service) for details

In [23]:
ground_check = check_grounding_client.check_grounding(
    request = discoveryengine.CheckGroundingRequest(
        grounding_config = check_grounding_client.grounding_config_path(
            project = PROJECT_ID,
            location = 'global',
            grounding_config = "default_grounding_config",
        ),
        answer_candidate = answer,
        facts = [
            discoveryengine.GroundingFact(
                fact_text = content_chunks[vector_db[match[0]][0]],
                attributes = {'author': 'MLB', 'chunk_id': vector_db[match[0]][0]}
            )
            for match in matches
        ],
        grounding_spec = discoveryengine.CheckGroundingSpec(
            citation_threshold = 0.6,
            #enable_anti_citation = True,
            #anti_citation_threshold = 0.75
        ),
    )
)

In [24]:
ground_check

support_score: 0.826414347
cited_chunks {
  chunk_text: "# 2.00-THE PLAYING FIELD\n\n## 2.02 Home Base\n\nHome base shall be marked by a five-sided slab of whitened rubber. It shall be a 17-inch square with two of the corners removed so that one edge is 17 inches long, two adjacent sides are 8\\frac{1}{2} inches and the remaining two sides are 12 inches and set at an angle to make a point."
  source: "0"
}
cited_chunks {
  chunk_text: "# 2.00-THE PLAYING FIELD\n\n## 2.02 Home Base\n\nIt shall be set in the ground with the point at the intersection of the lines extending from home base to first base and to third base; with the 17-inch edge facing the pitcher\'s plate, and the two 12-inch edges coinciding with the first and third base lines. The top edges of home base shall be beveled and the base shall be fixed in the ground level with the ground surface. (See drawing D in Appendix 2.) 3"
  source: "1"
}
cited_chunks {
  chunk_text: "# APPENDICES\n\n## Appendix 2\n\nDiagram No. 2 Layout

### Examine Check Grounding Results Phrase-by-Phrase

In [25]:
overview = f"**Grounding Support Score (0-1):** {ground_check.support_score:.2f}\n|Answer Phrases|Citations|\n|---|---|\n"
all_ref_chunks = []
for claim in ground_check.claims:
    citations = [c for c in claim.citation_indices] if hasattr(claim, 'citation_indices') else []
    sources = [ground_check.cited_chunks[citation].source for citation in citations]
    ref_chunks = [vector_db[matches[int(source)][0]][0] for source in sources]
    if ref_chunks: all_ref_chunks += ref_chunks
    overview += f"|{claim.claim_text}|{ref_chunks}|\n"
overview += '-'*80 +'\n'    
for chunk_id in sorted(list(set(all_ref_chunks))):
    overview += f"**Chunk = {chunk_id}:**\n\n{content_chunks[chunk_id]}\n\n" + '-'*80 +'\n'

Markdown(overview.replace('#', ''))

**Grounding Support Score (0-1):** 0.83
|Answer Phrases|Citations|
|---|---|
|The dimensions of a base depend on whether it's home base or the other bases.|[]|
|* **Home Base:**  It is a five-sided slab with:|[]|
|* One edge of 17 inches|['c38', 'c39', 'c837']|
|* Two adjacent sides of 8 1/2 inches|['c38']|
|* Two remaining sides of 12 inches.|['c38']|
|* **First, Second, and Third Base:** They are white canvas or rubber-covered bags that are:|['c40']|
|* 18 inches square|[]|
|* 3 to 5 inches thick.|['c40']|
--------------------------------------------------------------------------------
**Chunk = c38:**

 2.00-THE PLAYING FIELD

 2.02 Home Base

Home base shall be marked by a five-sided slab of whitened rubber. It shall be a 17-inch square with two of the corners removed so that one edge is 17 inches long, two adjacent sides are 8\frac{1}{2} inches and the remaining two sides are 12 inches and set at an angle to make a point.

--------------------------------------------------------------------------------
**Chunk = c39:**

 2.00-THE PLAYING FIELD

 2.02 Home Base

It shall be set in the ground with the point at the intersection of the lines extending from home base to first base and to third base; with the 17-inch edge facing the pitcher's plate, and the two 12-inch edges coinciding with the first and third base lines. The top edges of home base shall be beveled and the base shall be fixed in the ground level with the ground surface. (See drawing D in Appendix 2.) 3

--------------------------------------------------------------------------------
**Chunk = c40:**

 Rule 2.03 to 2.05

 2.03 The Bases

First, second and third bases shall be marked by white canvas or rubber-covered bags, securely attached to the ground as indicated in Diagram 2. The first and third base bags shall be entirely within the infield. The second base bag shall be centered on second base. The bags shall be 18 inches square, not less than three nor more than five inches thick, and filled with soft material.

--------------------------------------------------------------------------------
**Chunk = c837:**

 APPENDICES

 Appendix 2

Diagram No. 2 Layout at Home Plate, 1st, 2nd, and 3rd Bases 18" A 18" 90° LAYOUT AT SECOND BASE FOR LAYOUT AT PITCHER'S PLATE SEE DIAGRAM NO. 3 90° 6" 17" 6" D E 3'0" 3'0" 4'0" 4 C 43" LAYOUT AT HOME BASE DIAGRAM NO. 2 LEGEND A 1st, 2nd, 3rd BASES BATTER'S BOX B B C CATCHER'S BOX D HOME BASE E PITCHER'S PLATE Rev2023RW 161 90° 4 FOUL LINE LAYOUT AT FIRST BASE

--------------------------------------------------------------------------------


## Ground with Google Search

Now you can add the `tools` keyword arg with a `grounding` tool of `grounding.GoogleSearchRetrieval()` to instruct the LLM to first perform a Google Search with the prompt, then construct an answer based on the web search results.

The search queries and [Search Entry Point](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/grounding-search-entry-points) are available for each `Candidate` in the response. The helper `function print_grounding_response()` prints the response text with citations.

**Without grounding**

In [38]:
question = "What are the dimensions of a base?"

response = llm.generate_content(question)

display(Markdown(response.text))

The term "base" can refer to many things depending on the context. Please provide more information about what you're referring to, such as:

* **Geometry:** Are you talking about the base of a triangle, rectangle, pyramid, or some other geometric shape?
* **Construction:** Are you asking about the base of a building, a bridge, or some other structure?
* **Other:** Is there another context for the term "base" that you have in mind?

Once I have a better understanding of your question, I can give you a more specific answer about the dimensions of a base. 


**Ground with Google Search**

In [41]:
tool = Tool.from_google_search_retrieval(grounding.GoogleSearchRetrieval())

response = llm.generate_content(question, tools=[tool])

print_grounding_response(response)

A baseball base is a square with dimensions of 15 inches by 15 inches (38.1 cm x 38.1 cm). [1]
 This has recently changed as of September 2022, where the bases were increased to 18 inches square. [2]
  The height of a baseball base can vary from 3 to 5 inches (7.62-12.7 cm). [1]
 

----
## Grounding Sources

**Web Search Queries:** ['what are the dimensions of a base']

**Search Entry Point:**
 <style>
.container {
  align-items: center;
  border-radius: 8px;
  display: flex;
  font-family: Google Sans, Roboto, sans-serif;
  font-size: 14px;
  line-height: 20px;
  padding: 8px 12px;
}
.chip {
  display: inline-block;
  border: solid 1px;
  border-radius: 16px;
  min-width: 14px;
  padding: 5px 16px;
  text-align: center;
  user-select: none;
  margin: 0 8px;
  -webkit-tap-highlight-color: transparent;
}
.carousel {
  overflow: auto;
  scrollbar-width: none;
  white-space: nowrap;
  margin-right: -12px;
}
.headline {
  display: flex;
  margin-right: 4px;
}
.gradient-container {
  position: relative;
}
.gradient {
  position: absolute;
  transform: translate(3px, -9px);
  height: 36px;
  width: 9px;
}
@media (prefers-color-scheme: light) {
  .container {
    background-color: #fafafa;
    box-shadow: 0 0 0 1px #0000000f;
  }
  .headline-label {
    color: #1f1f1f;
  }
  .chip {
    background-color: #ffffff;
    border-color: #d2d2d2;
    color: #5e5e5e;
    text-decoration: none;
  }
  .chip:hover {
    background-color: #f2f2f2;
  }
  .chip:focus {
    background-color: #f2f2f2;
  }
  .chip:active {
    background-color: #d8d8d8;
    border-color: #b6b6b6;
  }
  .logo-dark {
    display: none;
  }
  .gradient {
    background: linear-gradient(90deg, #fafafa 15%, #fafafa00 100%);
  }
}
@media (prefers-color-scheme: dark) {
  .container {
    background-color: #1f1f1f;
    box-shadow: 0 0 0 1px #ffffff26;
  }
  .headline-label {
    color: #fff;
  }
  .chip {
    background-color: #2c2c2c;
    border-color: #3c4043;
    color: #fff;
    text-decoration: none;
  }
  .chip:hover {
    background-color: #353536;
  }
  .chip:focus {
    background-color: #353536;
  }
  .chip:active {
    background-color: #464849;
    border-color: #53575b;
  }
  .logo-light {
    display: none;
  }
  .gradient {
    background: linear-gradient(90deg, #1f1f1f 15%, #1f1f1f00 100%);
  }
}
</style>
<div class="container">
  <div class="headline">
    <svg class="logo-light" width="18" height="18" viewBox="9 9 35 35" fill="none" xmlns="http://www.w3.org/2000/svg">
      <path fill-rule="evenodd" clip-rule="evenodd" d="M42.8622 27.0064C42.8622 25.7839 42.7525 24.6084 42.5487 23.4799H26.3109V30.1568H35.5897C35.1821 32.3041 33.9596 34.1222 32.1258 35.3448V39.6864H37.7213C40.9814 36.677 42.8622 32.2571 42.8622 27.0064V27.0064Z" fill="#4285F4"/>
      <path fill-rule="evenodd" clip-rule="evenodd" d="M26.3109 43.8555C30.9659 43.8555 34.8687 42.3195 37.7213 39.6863L32.1258 35.3447C30.5898 36.3792 28.6306 37.0061 26.3109 37.0061C21.8282 37.0061 18.0195 33.9811 16.6559 29.906H10.9194V34.3573C13.7563 39.9841 19.5712 43.8555 26.3109 43.8555V43.8555Z" fill="#34A853"/>
      <path fill-rule="evenodd" clip-rule="evenodd" d="M16.6559 29.8904C16.3111 28.8559 16.1074 27.7588 16.1074 26.6146C16.1074 25.4704 16.3111 24.3733 16.6559 23.3388V18.8875H10.9194C9.74388 21.2072 9.06992 23.8247 9.06992 26.6146C9.06992 29.4045 9.74388 32.022 10.9194 34.3417L15.3864 30.8621L16.6559 29.8904V29.8904Z" fill="#FBBC05"/>
      <path fill-rule="evenodd" clip-rule="evenodd" d="M26.3109 16.2386C28.85 16.2386 31.107 17.1164 32.9095 18.8091L37.8466 13.8719C34.853 11.082 30.9659 9.3736 26.3109 9.3736C19.5712 9.3736 13.7563 13.245 10.9194 18.8875L16.6559 23.3388C18.0195 19.2636 21.8282 16.2386 26.3109 16.2386V16.2386Z" fill="#EA4335"/>
    </svg>
    <svg class="logo-dark" width="18" height="18" viewBox="0 0 48 48" xmlns="http://www.w3.org/2000/svg">
      <circle cx="24" cy="23" fill="#FFF" r="22"/>
      <path d="M33.76 34.26c2.75-2.56 4.49-6.37 4.49-11.26 0-.89-.08-1.84-.29-3H24.01v5.99h8.03c-.4 2.02-1.5 3.56-3.07 4.56v.75l3.91 2.97h.88z" fill="#4285F4"/>
      <path d="M15.58 25.77A8.845 8.845 0 0 0 24 31.86c1.92 0 3.62-.46 4.97-1.31l4.79 3.71C31.14 36.7 27.65 38 24 38c-5.93 0-11.01-3.4-13.45-8.36l.17-1.01 4.06-2.85h.8z" fill="#34A853"/>
      <path d="M15.59 20.21a8.864 8.864 0 0 0 0 5.58l-5.03 3.86c-.98-2-1.53-4.25-1.53-6.64 0-2.39.55-4.64 1.53-6.64l1-.22 3.81 2.98.22 1.08z" fill="#FBBC05"/>
      <path d="M24 14.14c2.11 0 4.02.75 5.52 1.98l4.36-4.36C31.22 9.43 27.81 8 24 8c-5.93 0-11.01 3.4-13.45 8.36l5.03 3.85A8.86 8.86 0 0 1 24 14.14z" fill="#EA4335"/>
    </svg>
    <div class="gradient-container"><div class="gradient"></div></div>
  </div>
  <div class="carousel">
    <a class="chip" href="https://vertexaisearch.cloud.google.com/grounding-api-redirect/AZnLMfxVUNeYD9nogVwDZy6w6UBPvSrYnlrjYMB5XOTCY4UJ2RChNfr84iqpzkl5Rvk6ZHzQLPwBBr4IIBduiyFTdCZXPjjK1uXuu8hhm-NoGWLYK-k0xljdh1UZXuqcDJ0Ybsnc8kLMs9q-fxRPpx5ihbVImPE_1_ubh0yIjhr9wOMcHGfNoPs2SHWZdIWQZC8lghlNc0QvEScxa20MmNBIu4yi3Hec">what are the dimensions of a base</a>
  </div>
</div>

### Grounding Chunks
1. [dimensions.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AZnLMfx3byHLb72knxQV1uBQnFR7X3mLKzQUXjZ3m3R_y2TWCTidbIbhcGh0Y65lgmsoBnsRrW0BvyN_kGOU5_SRUS42dLe2ARgD9YkSPsVhWSfd90T9d8P94RFsgT2cmAzW4vYwfQPRYEtRbeVRDQ==)
2. [mlb.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AZnLMfxYT_Swj0xd-KBMkjQGivYTOE7P3b6sWkYjfGwJjern6oivzHkjL8152ZLNc6deNkdAqL0bvRQu8KH6owgORv0J7EEUQfoovT_wmi-A2s_p5dLLTdFpgT7dG0NunkUKCtS-gGkL4Rlx)


In [42]:
response
# response.to_dict()

candidates {
  content {
    role: "model"
    parts {
      text: "A baseball base is a square with dimensions of 15 inches by 15 inches (38.1 cm x 38.1 cm). This has recently changed as of September 2022, where the bases were increased to 18 inches square.  The height of a baseball base can vary from 3 to 5 inches (7.62-12.7 cm). \n"
    }
  }
  avg_logprobs: -0.37520906294899425
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
    probability_score: 0.099609375
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.114257812
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
    probability_score: 0.349609375
    severity: HARM_SEVERITY_LOW
    severity_score: 0.236328125
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
    probability_score: 0.129882812
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.0727539062
  }
  s